# Bayesian Cross Sectional Models

This notebook investigates the fit and parameter estimates of Bayesian models. We try the following three distributions:
- Pareto
- Weibull
- Generalised Pareto

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import dill
import os

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Self made modules
from thesis_tools.utils.data import *
from thesis_tools.statistical_tests.tests import *
from thesis_tools.models.bayesian_univariate_cross_sectional import *
from thesis_tools.models.bayesian_multivariate_cross_sectional import *

### Estimation

In [3]:
df = read_panel_data(observations_threshold=20)
len(df)

250

In [4]:
def train_or_retrieve_model(panel_df:pd.DataFrame, group:str, year:int, model_type:str, retrain_if_saved=False):
    model_name = f"{model_type}_{group}_{year}"
    model_path = f"../../Stored_Models/bayesian_univariate_cross_sectional/{model_name}.pkl"
    if os.path.exists(model_path) and not retrain_if_saved:
        with open(model_path, "rb") as f:
            model = dill.load(f)
    else:
        y_data = np.array(panel_df[(panel_df['year'] == year) & (panel_df['group'] == group)]['net_worth'].iloc[0])
        if model_type == 'Pareto':
            model = Pareto_One_Stage(y_data)
        elif model_type == 'Weibull':
            model = Weibull_One_Stage(y_data)
        elif model_type == 'GeneralisedPareto':
            model = GeneralisedPareto_One_Stage(y_data)
        model.fit()
        with open(model_path, "wb") as f:
            dill.dump(model, f)
    return model

In [5]:
# find all combinations of group and year
groups = df['group'].unique()
for group in groups:
    years = df[df['group'] == group]['year'].unique()
    for year in years:
        for model_type in ['Pareto', 'Weibull', 'GeneralisedPareto']:
            train_or_retrieve_model(df, group, year, model_type, retrain_if_saved=False)
            print(f"Trained model for {group} in {year} using {model_type} distribution")

Trained model for Alps in 2013 using Pareto distribution
Trained model for Alps in 2013 using Weibull distribution
Trained model for Alps in 2013 using GeneralisedPareto distribution
Trained model for Alps in 2014 using Pareto distribution
Trained model for Alps in 2014 using Weibull distribution
Trained model for Alps in 2014 using GeneralisedPareto distribution
Trained model for Alps in 2015 using Pareto distribution
Trained model for Alps in 2015 using Weibull distribution
Trained model for Alps in 2015 using GeneralisedPareto distribution
Trained model for Alps in 2016 using Pareto distribution
Trained model for Alps in 2016 using Weibull distribution
Trained model for Alps in 2016 using GeneralisedPareto distribution
Trained model for Alps in 2017 using Pareto distribution
Trained model for Alps in 2017 using Weibull distribution
Trained model for Alps in 2017 using GeneralisedPareto distribution
Trained model for Alps in 2018 using Pareto distribution
Trained model for Alps in 20

### Evaluation

#### Mean wealth prediction

In [6]:
mean_wealth = {}
years = df['year'].unique()
for year in years:
    if year != 2018: # Only 2018 for now
        continue
    mean_wealth[year] = {}
    for group in groups:
        print(f"Calculating mean wealth for {group} in {year}")
        model_pareto = train_or_retrieve_model(df, group, year, 'Pareto')
        model_weibull = train_or_retrieve_model(df, group, year, 'Weibull')
        model_generalisedpareto = train_or_retrieve_model(df, group, year, 'GeneralisedPareto')
        post_pred_pareto = model_pareto.posterior_predictive(progressbar=False)
        post_pred_weibull = model_weibull.posterior_predictive(progressbar=False)
        post_pred_generalisedpareto = model_generalisedpareto.posterior_predictive(progressbar=False)
        mean_data = np.array(df[(df['year'] == year) & (df['group'] == group)]['net_worth'].iloc[0]).mean()
        mean_wealth[year][group] = {
            'mean_data': mean_data,
            'mean_pareto': post_pred_pareto.mean(),
            'mean_weibull': post_pred_weibull.mean(),
            'mean_generalisedpareto': post_pred_generalisedpareto.mean()
        }

Sampling: [y]


Calculating mean wealth for Alps in 2018


Sampling: [y]


Sampling: [y]


Sampling: [y]


...

Sampling: [y]


Sampling: [y]


Calculating mean wealth for U.S. in 2018


Sampling: [y]


Sampling: [y]


In [11]:
mean_predictions_df = pd.DataFrame(mean_wealth[2018]).T

In [12]:
mean_predictions_df

,mean_data,mean_pareto,mean_weibull,mean_generalisedpareto
Alps,3.802222,94.405551,3.774346,4.753531
Asian Islands,3.167164,8.606723,3.100595,4.017254
Australia,2.741860,9.993671,2.726677,3.471403
Brazil,4.200000,27.956026,4.054421,5.264703
British Islands,3.826984,24.032562,3.803741,4.858240
Canada,3.228261,15.887571,3.143112,4.346145
China,3.310023,5.323206,3.087295,4.014662
France,7.438636,159.290223,6.668744,9.632544
Germany,4.704878,322.773878,4.658608,5.320438
India,3.698319,17.356010,3.610512,4.198256
